### Algorithm

1. Read the dataset.
2. Reshape the dataset
2. Split the dataset to:
    * 70% training (only contain normal data),
    * 10% validation (contain normal and anomaly data), and
    * 20% testing dataset (contain normal and anomaly data)
3. Create autoencoder model.
4. Train the model using training dataset.
5. Validate the model using validation dataset.
6. Test the model using testing dataset.

### Requirements

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score


### Hyperparameters

In [ ]:
window_size = 25
value_range = (0, 40000)

test_size = 0.25
random_state = 11

# Define model's hyperparameters
hidden_size = 64
output_size = 1
num_epochs = 4000
batch_size = 32
learning_rate = 0.001

### Reading dataset

In [ ]:
# Read the dataset
df = pd.read_csv('/nas.dbms/mahendra.data/Documents/annotation/NAB/data/realKnownCause/nyc_taxi.csv',low_memory=False)
df['timestamp'] = pd.to_datetime(df['timestamp'])


# The times of anomaly events (Ground Truth)
anomaly_points = [
        [
            "2014-11-01 06:00:00.000000",
            "2014-11-02 06:00:00.000000"
        ],
        [
            "2014-11-27 06:00:00.000000",
            "2014-11-28 06:00:00.000000"
        ],
        [
            "2014-12-25 06:00:00.000000",
            "2014-12-26 06:00:00.000000"
        ],
        [
            "2014-12-31 06:00:00.000000",
            "2015-01-01 06:00:00.000000"
        ],
        [
            "2015-01-26 16:00:00.000000",
            "2015-01-28 05:00:00.000000"
        ]
]

# Normal label: 0, Anomaly label: 1
df['anomaly'] = 0  # Set default values
for start, end in anomaly_points:
    df.loc[((df['timestamp'] >= start) & (df['timestamp'] <= end)), 'anomaly'] = 1

In [ ]:
print(df.head())
print("\nDataset size:")
print(df.shape)
print("\nDataset distribution:")
print(df['anomaly'].value_counts())

### Split training and testing dataset

In [ ]:
def plot_anomaly(df, label='anomaly', plot_item='value', time_col='timestamp', scatter=True):
    # Plot function
    plotter =  plt.scatter if scatter else plt.plot

    # Plot sections
    plt.close()
    plt.figure().set_figwidth(20)

    color = {0: 'blue', 1: 'red'}
    for i, c in color.items():
        data = df[df[label] == i].copy(deep=True)  # Select data
        data.index = data[time_col]  # Change index
        plotter(data.index, data[plot_item], color=c)  # Plot

    plt.show()


def plot_cm(cm):
    plt.close()
    # plt.figure(figsize=(8,8))
    # sns.set(font_scale = 1.5)

    ax = sns.heatmap(
        cm, # confusion matrix 2D array 
        annot=True, # show numbers in the cells
        fmt='d', # show numbers as integers
        xticklabels=[0,1],
        yticklabels=[0,1]
    )

    ax.set_xlabel("Predicted")
    ax.set_ylabel("Actual")
    plt.show()

In [ ]:
cut = 8700
df_train = df[:cut]  # Select three sections of anomaly for training
plot_anomaly(df_train)

In [ ]:
df_test = df[8700:]  # Select two sections of anomaly for testing
plot_anomaly(df_test)

### Reshape datasets

In [ ]:
def create_dataset(df, window_size, value_range):
    X, y = [], []
    for i in range(df.shape[0]-window_size):
        feature = df['value'].iloc[i:i+window_size]
        target = df['anomaly'].iloc[i:i+window_size]
        X.append(feature)
        y.append(target)

    # Convert to numpy
    X = np.array(X)
    y = np.array(y)

    # Normalization
    X = (X - value_range[0]) / (value_range[1] - value_range[0])

    # Calculate the median of the labels
    y = np.round(np.mean(y, axis=1))

    return X, y


# Create training and testing dataset
X_train, y_train = create_dataset(df_train, window_size, value_range)
X_test, y_test = create_dataset(df_test, window_size, value_range)

In [ ]:
print("Training dataset")
print("X shape:", X_train.shape)
print("y shape:", y_train.shape)
print("Anomaly distribution:")
print(pd.Series(y_train).value_counts())

print("\nTesting dataset")
print("X shape:", X_test.shape)
print("y shape:", y_test.shape)
print("Anomaly distribution:")
print(pd.Series(y_test).value_counts())

### Training and Testing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Define the MLP model
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fcOut = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.relu3(out)
        out = self.fcOut(out)
        return out


def predict(model, features_tensor):
    with torch.no_grad():
        model.eval()
        train_outputs = model(features_tensor)
        return torch.round(torch.sigmoid(train_outputs)).squeeze().cpu().numpy()


# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)

# Define input size
input_size = X_train.shape[1]

# Create the MLP model
model = MLPClassifier(input_size, hidden_size, output_size).to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(X_train_tensor, y_train_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

progress_bar = tqdm(range(num_epochs),ncols=150)
progress_bar.set_description("Training")
for epoch in progress_bar:
    # Set the model in training mode
    model.train()

    for batch_features, batch_labels in dataloader:
        # Move batch to GPU
        batch_features = batch_features.to(device)
        batch_labels = batch_labels.to(device)
        
        # Forward pass
        outputs = model(batch_features)

        # Calculate loss
        loss = criterion(outputs.squeeze(), batch_labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Print the loss after each epoch
    progress_bar.set_postfix(Loss="%.4f" % loss.item())


# Train prediction
train_predictions = predict(model, X_train_tensor)

# Test prediction
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
test_predictions = predict(model, X_test_tensor)


### Matrix

In [ ]:
def show_matrix(y, y_pred):
    ps = precision_score(y, y_pred, pos_label=1, labels=[0, 1])
    print("precision_score: %.2f" % ps)

    rs = recall_score(y, y_pred, pos_label=1, labels=[0, 1])
    print("recall_score: %.2f" % rs)

    f1 = f1_score(y, y_pred, pos_label=1, labels=[0, 1])
    print("f1_score: %.2f" % f1)

    cm = confusion_matrix(y, y_pred, labels=[0, 1])
    plot_cm(cm)

In [ ]:
show_matrix(y_train, train_predictions)

In [ ]:
show_matrix(y_test, test_predictions)